---

In [1]:
from pathlib import Path

path = Path().absolute()

In [2]:
cd ../

c:\Users\c10nGp4\OneDrive\Documents\GitHub\imbalance-multi-classification


In [3]:
import pickle
import time

---

### Instantiate Classifier
---

In [4]:
from pipeline.classification import Classification

clf = Classification()

2023-06-06 14:53:17 INFO: DOWNLOAD STANZA MODEL


2023-06-06 14:53:17 INFO: Downloading default packages for language: id (Indonesian) ...
2023-06-06 14:53:18 INFO: File exists: C:\Users\c10nGp4\stanza_resources\id\default.zip
2023-06-06 14:53:20 INFO: Finished downloading models and saved to C:\Users\c10nGp4\stanza_resources.
2023-06-06 14:53:20 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-06-06 14:53:20 INFO: LOAD STANZA PIPELINE: tokenize,mwt,pos,lemma


2023-06-06 14:53:27 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2023-06-06 14:53:27 INFO: Using device: cuda
2023-06-06 14:53:27 INFO: Loading: tokenize
2023-06-06 14:53:30 INFO: Loading: mwt
2023-06-06 14:53:30 INFO: Loading: pos
2023-06-06 14:53:30 INFO: Loading: lemma
2023-06-06 14:53:30 INFO: Done loading processors!


### Load Data
---

In [73]:
import pandas as pd

# Load dataset
training_set_df = pd.read_csv(path / "assets/datasets/training-set-1.csv", delimiter=";")
# testing_set_df = pd.read_csv(path / "assets/datasets/testing-set-1.csv", delimiter=";")

# # Select specific categories
# # training_set_df = training_set_df[training_set_df["targets"].isin(["kaget","takut"])]
# # testing_set_df = testing_set_df[testing_set_df["targets"].isin(["kaget","takut"])]

# Get X and y from dataset
X_train = list(training_set_df["texts"])
# y_train = list(training_set_df["targets"])

# X_test = list(testing_set_df["texts"])
# y_test = list(testing_set_df["targets"])

### Data Preparation
---

In [74]:
from pipeline.pos_filter import POS

# Text Preprocessing
X_train = clf.text_preprocessing_pipeline.transform(X_train)
X_test = clf.text_preprocessing_pipeline.transform(X_test)

# Feature Selection

# emotion classification
clf.feature_selection_pipeline.named_steps["pos_filter"].set_params(**{"pos": POS - set(["ADP","AUX","CCONJ","DET","INTJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"])})

# aspect classification
# clf.feature_selection_pipeline.named_steps["pos_filter"].set_params(**{"pos": set(["NOUN","PROPN"])})

# extract word features
clf.feature_selection_pipeline.named_steps["document_transformer"].set_params(**{"feat_attrs": ["text"]})

X_train = clf.feature_selection_pipeline.transform(X_train)
X_test = clf.feature_selection_pipeline.transform(X_test)

2023-06-06 15:08:44 INFO: TEXT CLEANING
2023-06-06 15:08:46 INFO: TOKENIZE, MWT, POS, LEMMA
2023-06-06 15:09:43 INFO: TEXT CLEANING
2023-06-06 15:09:43 INFO: TOKENIZE, MWT, POS, LEMMA
2023-06-06 15:09:57 INFO: POS REMOVAL
2023-06-06 15:09:57 INFO: STOPWORD REMOVAL
2023-06-06 15:09:58 INFO: DOCUMENT TRANSFORMER
2023-06-06 15:09:58 INFO: POS REMOVAL
2023-06-06 15:09:58 INFO: STOPWORD REMOVAL
2023-06-06 15:09:58 INFO: DOCUMENT TRANSFORMER


### Hyper-parameters tuning
---

In [ ]:
# from datetime import timedelta

# n_iter = 10
# n_splits = 5
# train_size = 0.8
# n_jobs = 1
# verbose = 3

# param_distributions = {
#     "tfidfvectorizer__ngram_range": ((1, 1), (1, 2), (2, 2)),
#     "tfidfvectorizer__min_df": (0.01, 1, 3, 5, 10),
#     "tfidfvectorizer__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
#     "tfidfvectorizer__norm": (None, "l1", "l2"),
#     "tfidfvectorizer__sublinear_tf": (True, False),
#     "linearsvc__penalty": ("l1","l2"),
#     "linearsvc__loss": ("hinge","squared_hinge"),
#     "linearsvc__dual": (True, False),
#     "linearsvc__tol": (0.0001,),
#     "linearsvc__C": (0.01, 0.1, 1, 10, 100),
#     "linearsvc__multi_class": ("ovr",),
#     "linearsvc__fit_intercept": (True, False),
#     "linearsvc__intercept_scaling": (1.0,),
#     "linearsvc__class_weight": (None, "balanced", {"kaget": 1, "cinta": 3, "takut": 4, "marah": 5, "gembira": 10, "sedih": 11}),
#     "linearsvc__max_iter": (1000,)
# }

# randomized_search, estimation = clf.tuning(X_train, y_train, param_distributions, n_iter, n_splits, train_size, n_jobs, verbose)
# cv_results_df = pd.DataFrame(randomized_search.cv_results_)


In [ ]:
# cv_results_df

In [ ]:
# print(f'Fitted {randomized_search.n_splits_} folds of {len(cv_results_df)} candidates, finished in {str(timedelta(seconds=estimation))}.')
# print(f"Best score: {randomized_search.best_score_}")
# print("Best hyper-parameters:")
# randomized_search.best_params_

### DO Update classification pipeline
---

In [ ]:
# clf.classification_pipeline = randomized_search.best_estimator_

### OR Re-train Model
---

#### Apply the best hyper-parameters
---

In [ ]:
# clf.classification_pipeline.set_params(**randomized_search.best_params_)

#### Train model
---

In [ ]:
# clf.train_preprocessed(X_train, y_train)

### Save model
---

In [ ]:
# clf.to_disk(path / "assets/models/")

### Load model
---

In [ ]:
# filepath = path / "assets/models/model.2023.05.24.14.40.25.798494.pickle"
# clf.from_disk(filepath)

### Test model
---

In [ ]:
# y_pred = clf.test_preprocessed(X_test)

### Evaluation
---

In [ ]:
# accuracy, mcc = clf.score(y_test, y_pred)

# print("Accuracy:", accuracy)
# print("MCC:", mcc)

### Confusion Matrix
---

In [ ]:
# from sklearn.metrics import ConfusionMatrixDisplay

# normalize = None
# # normalize="true"

# ConfusionMatrixDisplay.from_predictions(y_test, y_pred, normalize=normalize, cmap="YlGn")

## Analysis
---

In [ ]:
# clf.classification_pipeline.named_steps["tfidfvectorizer"].get_feature_names_out()

### Examples
---

#### Get random samples
---

In [ ]:
# from random import sample

In [ ]:
# sample([x for x in X_train if len(x) <= 50 and "gugup" in x], 5)

#### Example of a dataset
---

In [ ]:
X_temp = [
    "<b>Saya senang bisa melihat 🎆 bersamamu</b>",
    "Saya merasa benar-benar dirugikan dirinya!!!!",
    "Video menyeramkan lainnya: https://www.youtube.com/shorts/ZIl2iiHy1oM. ***JANGAN DILIHAT KALAU TIDAK BERANI***",
    "Saya merasa kesepian dan terisolasi akhir akhir ini",
    "saya merasa sangat sedih saya berharap saya mati",
    "saya merasa sedih hanya memikirkannya",
    "saya merasa patah hati dan sedih",
    "saya merasa sangat putus asa dan sedih",
    "Saya selalu merasa tidak diinginkan dan sedih",
    "saya merasa senang",
    "saya merasa cukup senang minggu ini",
    "saya sangat senang tapi saya merasa lelah",
    "saya merasakan cinta dan perhatian sekarang",
    "saya merasa disambut dan dicintai",
    "saya merasa dicintai",
    "saya merasa takut berada di dekat mereka",
    "saya merasa sangat ketakutan dan gugup",
    "saya merasa takut dan bodoh",
    "saya merasa takut",
    "saya sangat suka perasaan takut",
    "saya merasa seperti orang bodoh",
    "saya merasa bodoh tapi senang",
    "saya merasa sangat bodoh karena jujur",
    "saya merasa sangat bodoh dan malu",
    "saya merasa bodoh dan tidak diinginkan",
    "saya merasa kesal dan marah",
    "saya berteriak dengan perasaan sangat marah",
    "saya bahkan tidak bisa merasa marah karenanya",
    "saya merasa sedikit kesal hari ini",
    "saya bangun hari ini merasa kesal",
    "saya merasa sangat gemetar dan sedih",
    "saya menjadi bodoh merasa gugup"
]

y_temp = [
    "gembira",
    "marah",
    "takut",
    "sedih",
    "sedih",
    "sedih",
    "sedih",
    "sedih",
    "sedih",
    "gembira",
    "gembira",
    "gembira",
    "cinta",
    "cinta",
    "cinta",
    "takut",
    "takut",
    "takut",
    "takut",
    "takut",
    "sedih",
    "gembira",
    "sedih",
    "sedih",
    "sedih",
    "marah",
    "marah",
    "marah",
    "marah",
    "marah",
    "sedih",
    "sedih"
]

#### Split dataset
---

In [ ]:
from sklearnex.model_selection import train_test_split

X_temp_train, X_temp_test, y_temp_train, y_temp_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.2,
    random_state=42,
    stratify=y_temp
)

X_temp_train, X_temp_validation, y_temp_train, y_temp_validation = train_test_split(
    X_temp_train,
    y_temp_train,
    test_size=0.21,
    random_state=42,
    stratify=y_temp_train
)

In [ ]:
print(len(X_temp_train))

# for x in X_temp_train:
#     print(x)

for y in y_temp_train:
    print(y)

In [ ]:
print(len(X_temp_validation))

# for x in X_temp_validation:
#     print(x)

for y in y_temp_validation:
    print(y)

In [ ]:
print(len(X_temp_test))

# for x in X_temp_test:
#     print(x)

for y in y_temp_test:
    print(y)

#### Text Preprocessing
---

In [ ]:
X_temp_train_docs = clf.text_preprocessing_pipeline.transform(X_temp_train)
X_temp_validation_docs = clf.text_preprocessing_pipeline.transform(X_temp_validation)
X_temp_test_docs = clf.text_preprocessing_pipeline.transform(X_temp_test)

#### Feature Selection
---

In [ ]:
clf.feature_selection_pipeline.named_steps["pos_filter"].set_params(**{"pos": set(["ADJ","ADV","NOUN","PART","VERB"])})
clf.feature_selection_pipeline.named_steps["document_transformer"].set_params(**{"feat_attrs": ["lemma","upos"]})
X_temp_train_selected = clf.feature_selection_pipeline.transform(X_temp_train_docs)
X_temp_validation_selected = clf.feature_selection_pipeline.transform(X_temp_validation_docs)
X_temp_test_selected = clf.feature_selection_pipeline.transform(X_temp_test_docs)

In [ ]:
for i, doc in enumerate(X_temp_train_selected):
    print(", ".join(doc))

In [ ]:
for i, doc in enumerate(X_temp_validation_selected):
    print(", ".join(doc))

In [ ]:
for i, doc in enumerate(X_temp_test_selected):
    print(", ".join(doc))

#### Feature Extraction
---

In [ ]:
tfidfvectorizer_hyperparameters = {
    "ngram_range": (1, 1),
    "max_df": (1.0),
    "min_df": (1),
    "norm": None,
    "sublinear_tf": False
}
tfidfvectorizer = clf.classification_pipeline.named_steps["tfidfvectorizer"]
tfidfvectorizer.set_params(**tfidfvectorizer_hyperparameters)
tfidfvectorizer.fit(X_temp_train_selected)

In [ ]:
for i, zip_ in enumerate(
    zip(
        dict(
            sorted(
                tfidfvectorizer.vocabulary_.items(),
                key=lambda x: x[1]
            )
        ).items(),
        tfidfvectorizer.idf_
    )
):
    item, idf = zip_
    k, v = item
    df = 0

    for doc in X_temp_train_selected:
        if k in doc:
            df+=1

    print(i+1, k, df, round(idf, 5), sep="\t")
    # print(i+1, k, df, sep="\t")

In [ ]:
X_temp_train_features = tfidfvectorizer.transform(X_temp_train_selected)
X_temp_validation_features = tfidfvectorizer.transform(X_temp_validation_selected)
X_temp_test_features = tfidfvectorizer.transform(X_temp_test_selected)

#### Multi classification
---

In [ ]:
linearsvc_hyperparameters = {
    "penalty": "l2",
    "loss": "squared_hinge",
    "dual": False,
    "tol": 0.0001,
    "C": 0.01,
    "multi_class": "ovr",
    "fit_intercept": True,
    "intercept_scaling": 1.0,
    "class_weight": "balanced",
    "max_iter": 1000
}
linearsvc = clf.classification_pipeline.named_steps["linearsvc"]
linearsvc.set_params(**linearsvc_hyperparameters)
linearsvc.fit(X_temp_train_features, y_temp_train)

In [ ]:
y_temp_validation_pred = linearsvc.predict(X_temp_validation_features)
y_temp_test_pred = linearsvc.predict(X_temp_test_features)

#### Evaluation
---

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

In [ ]:
# test
print(accuracy_score(y_temp_validation, y_temp_validation_pred))
print(matthews_corrcoef(y_temp_validation, y_temp_validation_pred))

In [ ]:
# test
print(accuracy_score(y_temp_test, y_temp_test_pred))
print(matthews_corrcoef(y_temp_test, y_temp_test_pred))